In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle

In [29]:
#load the datasset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [30]:
##Drop irrelebvant data
df = df.drop(labels=['RowNumber','CustomerId','Surname'],axis=1)

In [31]:
##Encode categorical variables - Geography(OHE), Gender(LabelEncoder)
le = LabelEncoder()
df['Gender'] =le.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [32]:
##OHE on geography column
ohe = OneHotEncoder()
geo_encoder = ohe.fit_transform(df[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [33]:
##Concat the data and geo_encoded_df
df = df.drop('Geography',axis=1)
df = pd.concat([df,geo_encoded_df],axis=1)
df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [34]:
##Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(le,f)

with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(ohe,f)

In [35]:
##Divide into X and y
X=df.drop(['EstimatedSalary','Exited'],axis=1)
y=df['EstimatedSalary']

In [36]:
##Split to train and test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


##Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
with open('std_scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

ANN Regression

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [39]:
##ANN Model

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##HL1
    Dense(32,activation='relu'), ##HL2
    Dense(1) ##Output layer = Default is Linear Regression Activation function
]
)

In [40]:
##Optimiser creation with custom learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [41]:
##Compile the model - Here in regression we use mae as loss
#Here optimiser will have fixed learning rate
model.compile(optimizer=opt,loss="mean_absolute_error",metrics=['mae'])

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                768       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
## SEtup the tensorboard for logs during training
import datetime
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [44]:
##SEtup early stopping = to find best epoch count
earlystoppingcallback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
##Monitor the validation loss, wait for atleast 5 epochs, use the best weights obtained in an epoch as final

In [45]:
##Train the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tf_callback,earlystoppingcallback]
)

Epoch 1/100
250/250 [==============================] - 3s 8ms/step - loss: 81021.9844 - mae: 81021.9844 - val_loss: 50864.6719 - val_mae: 50864.6719
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 50037.6719 - mae: 50037.6719 - val_loss: 50285.6719 - val_mae: 50285.6719
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 49815.4062 - mae: 49815.4062 - val_loss: 50344.8672 - val_mae: 50344.8672
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 49676.2891 - mae: 49676.2891 - val_loss: 50318.4102 - val_mae: 50318.4102
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 49636.1719 - mae: 49636.1719 - val_loss: 50402.8594 - val_mae: 50402.8594
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 49576.6719 - mae: 49576.6719 - val_loss: 50295.4961 - val_mae: 50295.4961
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 49550.0312 - mae: 49550.0312 - 

In [46]:
##Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 12892), started 0:22:49 ago. (Use '!kill 12892' to kill it.)

In [48]:
##Evaluate model on test data

test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'Test Mae: {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50285.6719 - mae: 50285.6719
Test Mae: 50285.671875


In [49]:
model.save('regression_model.h5')

c:\Users\mrabi\OneDrive - Hewlett Packard Enterprise\Documents\DS\DL End to End Projects\ANN Classification\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
